In [21]:
#Setting up environment
import pandas as pd
import os
import numpy as np
from IPython.display import Math

from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
import matplotlib.pylab as plt
import seaborn as sns
from sklearn import metrics, svm,tree,preprocessing
from pprint import pprint
from sklearn.metrics import (auc, confusion_matrix, roc_curve, accuracy_score, precision_score)
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV
import xgboost
import shap


import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
df_processed= pd.read_csv("/Suzi fun files/QB course/QB_DS_FinalProject/data/processed/data_all_features.csv", index_col=[0])

In [3]:
X_test_df= pd.read_csv("/Suzi fun files/QB course/QB_DS_FinalProject/data/interim/X_test.csv", index_col=[0])
X_test_df.columns=df_processed.columns[:-1]
X_test = X_test_df.values

X_train_df= pd.read_csv("/Suzi fun files/QB course/QB_DS_FinalProject/data/interim/X_train.csv", index_col=[0])
X_train_df.columns=df_processed.columns[:-1]
X_train = X_train_df.values

y_train_df = pd.read_csv('/Suzi fun files/QB course/QB_DS_FinalProject/data/interim/y_train.csv', header=None, index_col=[0]);
y_train = y_train_df.iloc[:,0]

y_test_df = pd.read_csv('/Suzi fun files/QB course/QB_DS_FinalProject/data/interim/y_test.csv', header=None, index_col=[0]);
y_test = y_test_df.iloc[:,0]

In [13]:
shap.initjs()

train = xgboost.DMatrix(X_train, label=y_train)
test = xgboost.DMatrix(X_test, label=y_test)

model = xgboost.train({"learning_rate": 0.01},train, 100)

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(train)

# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.force_plot(explainer.expected_value, shap_values[0,:], df_processed.iloc[0,:-1])


In [15]:
shap.force_plot(explainer.expected_value, shap_values, X_train_df)

In [22]:
prediction = model.predict(test)
best_prediction = np.asarray([np.argmax(line) for line in prediction])
precision_score(y_test, best_prediction, average='macro')

0.31140350877192985